## **CIND 820 - Big Data Analytics Project**

In [204]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import svm, metrics 
from sklearn.model_selection import train_test_split # to apply train-test split
from sklearn.model_selection import cross_val_score, KFold # to apply k-fold cross-validation 
from sklearn.metrics import confusion_matrix, classification_report

**Create a data frame of the Room Occupancy dataset. Display the names of the columns. Check the data types of the attributes. Run a summary statistics of the dataset.**

In [158]:
# Read the CSV file into a data frame called Occupancy
Occupancy = pd.read_csv('Occupancy_Estimation.csv')

# Display the first five records of the data frame
Occupancy.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


In [159]:
# Get the column names
print("Column Names:", Occupancy.columns)

Column Names: Index(['Date', 'Time', 'S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light',
       'S2_Light', 'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound',
       'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR',
       'Room_Occupancy_Count'],
      dtype='object')


In [160]:
# Get the data types of each column
print("\nData Types of Each Column:")
print(Occupancy.dtypes)


Data Types of Each Column:
Date                     object
Time                     object
S1_Temp                 float64
S2_Temp                 float64
S3_Temp                 float64
S4_Temp                 float64
S1_Light                  int64
S2_Light                  int64
S3_Light                  int64
S4_Light                  int64
S1_Sound                float64
S2_Sound                float64
S3_Sound                float64
S4_Sound                float64
S5_CO2                    int64
S5_CO2_Slope            float64
S6_PIR                    int64
S7_PIR                    int64
Room_Occupancy_Count      int64
dtype: object


In [161]:
# Get the summary statistics
print("Summary Statistics:")
print(Occupancy.describe())

Summary Statistics:
            S1_Temp       S2_Temp       S3_Temp       S4_Temp      S1_Light  \
count  10129.000000  10129.000000  10129.000000  10129.000000  10129.000000   
mean      25.454012     25.546059     25.056621     25.754125     25.445059   
std        0.351351      0.586325      0.427283      0.356434     51.011264   
min       24.940000     24.750000     24.440000     24.940000      0.000000   
25%       25.190000     25.190000     24.690000     25.440000      0.000000   
50%       25.380000     25.380000     24.940000     25.750000      0.000000   
75%       25.630000     25.630000     25.380000     26.000000     12.000000   
max       26.380000     29.000000     26.190000     26.560000    165.000000   

          S2_Light      S3_Light      S4_Light      S1_Sound      S2_Sound  \
count  10129.00000  10129.000000  10129.000000  10129.000000  10129.000000   
mean      26.01629     34.248494     13.220259      0.168178      0.120066   
std       67.30417     58.400744  

**Scale the numeric features**

In [218]:
# Create the scaler
scaler = StandardScaler()

# Transform the numeric data
Occupancy_scaled = pd.DataFrame(scaler.fit_transform(Occupancy.iloc[:, 2:16]))

# Add the categorical data
Occupancy_scaled['S6_PIR'] = Occupancy['S6_PIR']
Occupancy_scaled['S7_PIR'] = Occupancy['S7_PIR']
Occupancy_scaled['Room_Occupancy_Count'] = Occupancy['Room_Occupancy_Count']

Occupancy_scaled.columns = ['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp',
                            'S1_Light', 'S2_Light', 'S3_Light', 'S4_Light',
                            'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound',
                            'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR', 'Room_Occupancy_Count']
Occupancy_scaled.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,-1.463033,-1.357775,-1.162334,-1.049684,1.873305,0.118627,0.321099,1.366226,-0.278432,0.262426,-0.237223,-0.363288,-0.354382,0.664469,0,0,1
1,-1.463033,-1.357775,-1.162334,-0.881342,1.873305,0.103769,0.321099,1.366226,2.405552,-0.262923,-0.237223,-0.363288,-0.354382,0.558817,0,0,1
2,-1.292255,-1.357775,-1.302764,-0.881342,1.873305,0.118627,0.321099,1.366226,0.826738,-0.037773,-0.188869,-0.363288,-0.354382,0.449864,0,0,1
3,-1.292255,-1.357775,-1.162334,-0.881342,1.873305,0.118627,0.321099,1.366226,0.763585,-0.075298,-0.140515,-0.114690,-0.354382,0.337609,0,0,1
4,-1.292255,-1.357775,-1.162334,-0.881342,1.873305,0.118627,0.338223,1.366226,0.037331,-0.225398,-0.237223,-0.363288,-0.354382,0.222053,0,0,1


**Split the data into training and testing sets**

In [219]:
# Define the predictor variables and the response variable
X = Occupancy_scaled.iloc[:, 0:16]
y = Occupancy_scaled['Room_Occupancy_Count']

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=104, shuffle=True)

**Use SVM for modeling**

In [221]:
# Create a SVM classifier
model = svm.SVC(kernel='rbf') # RBF Kernel

**Apply cross validation on the training set**

In [222]:
# Prepare the cross-validation procedure
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=1)

In [223]:
# Apply cross-validation on the training set
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kf, n_jobs=-1)

In [224]:
# Print accuracy
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
      (np.mean(scores), np.std(scores)))

Accuracy: 0.993 ,
Standard Deviations :0.003


In [225]:
# Evaluation metrics
print(f'Cross-Validation Results (Accuracy): {scores}')
print(f'Mean Accuracy: {scores.mean()}')

Cross-Validation Results (Accuracy): [0.99630086 0.99506782 0.99383477 0.98765432 0.9962963  0.99259259
 0.99012346 0.9962963  0.98888889 0.9962963 ]
Mean Accuracy: 0.9933351600675892


**Train the model on the full training set and evaluate on the test set** 

In [226]:
# Train the model using the training sets
model.fit(X_train, y_train)

SVC()

In [227]:
# Predict the response for test dataset
y_pred = model.predict(X_test)
print(y_pred[:10])

[0 0 0 2 0 0 0 0 2 3]


In [228]:
# Calculate the performance metrics for our logistic regression machine learning model
class_report = classification_report(y_test, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1679
           1       1.00      0.99      0.99        81
           2       0.98      0.98      0.98       127
           3       0.97      0.98      0.97       139

    accuracy                           1.00      2026
   macro avg       0.99      0.99      0.99      2026
weighted avg       1.00      1.00      1.00      2026



In [216]:
# Create the confusion matrix for the model
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

[[1679    0    0    0]
 [   0   80    0    1]
 [   0    0  124    3]
 [   0    0    3  136]]
